# PWV10:Analyse  Spectra and apply photometric selection from spectra saved in hdf5 for photometric night and View PWV

- The spectra and the transmission has been generated from **Fit_GP_PWV_timeseq-and-FetchSpectra-savehdf5.ipynb**
- The selection of photometric nights is done in **Analysesavehdf5-transmission.ipynb**

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2024-12-10 :
- Kernel @usdf **w_2024_41*
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310
- MUST RUN AT USDF

**Goal** : Show correlation holo /Merra

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os,re

#### Create output directory for figures and data

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV10_AnalyseApplyPhotomPWVPernightSpectra"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
pathdata = "data_PWV05_HoloFITLinearandGPPWVandFetchNightSpectra"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datapath_input = os.path.join(pathdata,"pernightspectra") 
if not os.path.exists(datapath_input):
    assert False

In [ ]:
pathdatain_selection = "data_PWV08_dataHoloAnalyseSelectPhotometricNightSpectra"
filename_photom_input ="selection_photomnights.csv"
fullfilename_photom_input = os.path.join(pathdatain_selection,filename_photom_input)

#### Imports

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

import numpy as np
from numpy.linalg import inv


from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

import scipy
from scipy.optimize import curve_fit,least_squares
from scipy.interpolate import RegularGridInterpolator
from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

# new color correction model
import pickle

from tqdm import tqdm

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

#### Butler access

- these access may change with time
- check my collections are found

#### Environnement for rubinsimphot
- Configure to access to rubinsimphot
- Note it is important to have the rubinsimphot path in the python path
- check the dm_version repository the newpythonpath 

In [ ]:
dm_version="repos_w_2025_36" 
machine_name = os.uname().nodename
print(machine_name)
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    #machine_name_notebook platform = 'dagoret-nb'
    print("Set environement for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),f"repos/{dm_version}/rubinsimphot/src")
    sys.path.append(newpythonpath)
    newpythonpath = os.path.join(os.getenv("HOME"),"rubin-user/RubinLSSTPhotometricCorrTuto/notebooks/lib")
    sys.path.append(newpythonpath)   
elif "dagoret-nb" in machine_name:
    print("Set environement for Rubin Platform at  USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),f"repos/{dm_version}/rubinsimphot/src")
    sys.path.append(newpythonpath)
    newpythonpath = os.path.join(os.getenv("HOME"),"rubin-user/RubinLSSTPhotometricCorrTuto/notebooks/lib")
    sys.path.append(newpythonpath)
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environnement named conda_py310")
else:
    print("Your current machine name is {machine_name}. Check your python environment")

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data.data_sets import  get_data_dir

#README.md        darksky.dat      filter_r.dat     hardware_g.dat   hardware_y.dat   lens3.dat        total_g.dat      total_y.dat
#README_SOURCE.md detector.dat     filter_u.dat     hardware_i.dat   hardware_z.dat   m1.dat           total_i.dat      total_z.dat
#atmos_10.dat     filter_g.dat     filter_y.dat     hardware_r.dat   lens1.dat        m2.dat           total_r.dat      version_info
#atmos_std.dat    filter_i.dat     filter_z.dat     hardware_u.dat   lens2.dat        m3.dat           total_u.dat
hardware_filenames = ["hardware_u.dat","hardware_g.dat","hardware_r.dat","hardware_i.dat","hardware_z.dat","hardware_y.dat"] 
filter_filenames = ["filter_u.dat","filter_g.dat","filter_r.dat","filter_i.dat","filter_z.dat","filter_y.dat" ]
total_filenames = ["total_u.dat","total_g.dat","total_r.dat","total_i.dat","total_z.dat","total_y.dat" ]
filter_tagnames = ["u","g","r","i","z","y"]
Filter_tagnames = ["U","G","R","I","Z","Y"]
filtercolor_tagnames = ["u-g","g-r","r-i","i-z","z-y"]
Filtercolor_tagnames = ["U-G","G-R","R-I","I-Y","Z-Y"]
filter_color = ["b","g","r","orange","grey","k"]
NFILT=len(filter_filenames)

WLMIN=300.
WLMAX=1100.
WLBIN=1.
NWLBIN=int((WLMAX-WLMIN)/WLBIN)
WL=np.linspace(WLMIN,WLMAX,NWLBIN)

In [ ]:
#FILTERWL: precalculated array containing center, boundaries and width of each filter.
#index 0 : minimum wavelength of filter border
#index 1 : minimum wavelength of filter border
#index 2 : center wavelength of filter
#index 3 : filter width


FILTERWL = np.array([[ 324.03003755,  402.12765957,  363.59690349,   78.09762203],
       [ 392.11514393,  561.32665832,  473.54069923,  169.21151439],
       [ 542.3028786 ,  700.50062578,  619.49926767,  158.19774718],
       [ 681.47684606,  827.65957447,  752.01084117,  146.18272841],
       [ 808.63579474,  932.79098874,  868.488419  ,  124.15519399],
       [ 914.76846058, 1044.93116395,  969.10570859,  130.16270338]])

FILTERWL_auxtel = np.array([[ 352.7 ,  395.9 ,  374.3 ,   43.2 ],
                     [ 387.6 ,  566.2 ,  476.9 ,  178.6 ],
                     [ 541.4 ,  715.5 ,  628.45,  174.1 ],
                     [ 673.3 ,  870.9 ,  772.1 ,  197.6 ],
                     [ 805.6 , 1090.7 ,  948.15,  285.1 ]])


F0 = 3631.0 # Jy 1, Jy = 10^{-23} erg.cm^{-2}.s^{-1}.Hz^{-1}
Jy_to_ergcmm2sm1hzm1 = 1e-23
DT = 30.0 # seconds
gel = 1.1
#hP = 6.62607015E-34 # J⋅Hz−1
hP = 6.626196E-27
A  = np.pi*642.3**2 # cm2  Reff=6.423 m
A_auxtel  = 9636.0 # cm2

#ZPT_cont =  2.5 \log_{10} \left(\frac{F_0 A \Delta T}{g_{el} h} \right)
ZPTconst = 2.5*np.log10(F0*Jy_to_ergcmm2sm1hzm1*A*DT/gel/hP)

In [ ]:
fdir = get_data_dir()
bandpass_inst = {}
path_rubin_sim_throughput = os.path.join(fdir, 'throughputs', 'baseline')
for index,filename in enumerate(hardware_filenames):
    fullfilename=os.path.join(path_rubin_sim_throughput,filename)
    arr= np.loadtxt(fullfilename)
    # interpolate  filter transmission
    ff = interpolate.interp1d(x=arr[:,0], y=arr[:,1],fill_value="extrapolate")
    fname = filter_tagnames[index]
    bandpass_inst[fname] = Bandpass(wavelen=WL,sb=ff(WL))

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(WL,bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(WL,bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total Rubin-LSST filter throughput")
plt.show()

### Load my stuff

In [ ]:
sys.path.append("../lib")
from libauxtelspectra import *
from libanaspectra import *

## Functions

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

### Spectro Hologram data

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

In [ ]:
version_results = "v1"
legendtag = {"v1" : "v3.1.0"}

In [ ]:
atmfilenamesdict = {"v1":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_atmosphere_20250912a_repomain_v1.npy",}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
#list(df_spec.columns)

### Remove spectra with red filter

In [ ]:
df_spec['FILTER'].unique()

In [ ]:
FLAG_REMOVE_FILTERS = True
if FLAG_REMOVE_FILTERS:
    df_spec=df_spec[df_spec["FILTER"] == 'empty']
    df_spec.reset_index(inplace=True)  

### Define if a target is faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:10]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]

### Add the Time in pd.datetime

#### UTC

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

### Compute relative time to Mid-night

In [ ]:
def GetTimeToMidNight(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(local_time.year,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
    local_time_midnight = datetime(local_time_new.year,local_time_new.month,local_time_new.day)
    dt_hour = (local_time_new -local_time_midnight).seconds/3600.

    # we took the previous night mid-night , must subtract 24H
    if dt_hour > 12.:
        dt_hour_new = (dt_hour - 24.)
    else:
        dt_hour_new = dt_hour
        
    return dt_hour_new

In [ ]:
df_spec["dt_midnight"] = df_spec.apply(GetTimeToMidNight,axis=1)

### Compute Date relative to January

In [ ]:
def GetDateToMidJanuary(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new)

In [ ]:
#df_spec["Time_january"] = df_spec.apply(GetDateToMidJanuary,axis=1)

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new),local_time.year 

In [ ]:
df_spec[["Time_january","Year"]] = df_spec.apply(GetDateToMidJanuaryAndYear,axis=1,result_type="expand")

In [ ]:
df_spec[["Time_january","Year"]]

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Apply Quality selection cuts

In [ ]:
def getSelectionCutOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) 
    return cut

In [ ]:
def getSelectionCutNoPolarOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] != "HD185975")
    return cut

In [ ]:
def getSelectionCutWithPolarOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
def getSelectionCut(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) 
    #(df_spec['EXPTIME'] > 20.) 
    #& (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
def getSelectionCutNoPolar(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) & \
    (df_spec["TARGET"] != "HD185975")
    #(df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
def getSelectionCutWithPolar(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) & \
    (df_spec["TARGET"] == "HD185975")
    #(df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
cut = getSelectionCut(df_spec) 
cut_nopolar = getSelectionCutNoPolar(df_spec) 
cut_nopolar_bright = getSelectionCutNoPolar(df_spec) & (~df_spec["isFaint"])
cut_nopolar_faint = getSelectionCutNoPolar(df_spec) & (df_spec["isFaint"])
cut_wthpolar = getSelectionCutWithPolar(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]
df_spec_np = df_spec[cut_nopolar] 
df_spec_np_b = df_spec[cut_nopolar_bright]
df_spec_np_f = df_spec[cut_nopolar_faint]
df_spec_wp = df_spec[cut_wthpolar]

In [ ]:
print("Total number of Spectra          : ",len(df_spec))
print("Number of selected Spectra       : ",len(df_spec_sel))
print("Number of selected Polars        : ",len(df_spec_wp))
print("Number of selected Non-Polars    : ",len(df_spec_np))
print("Number of selected Non-Polars Bright : ",len(df_spec_np_b))
print("Number of selected Non-Polars Faint  : ",len(df_spec_np_f))

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)
df_spec_np.reset_index(drop=True,inplace=True)
df_spec_wp.reset_index(drop=True,inplace=True) 
df_spec_np_b.reset_index(drop=True,inplace=True)
df_spec_np_f.reset_index(drop=True,inplace=True)

In [ ]:
#List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
print("Polar            :",len(df_spec_wp["TARGET"].unique()),"\t", df_spec_wp["TARGET"].unique()) 
print("Non Polar        :",len(df_spec_np["TARGET"].unique()),"\t" ,df_spec_np["TARGET"].unique())
print("Non Polar Bright :",len(df_spec_np_b["TARGET"].unique()),"\t" ,df_spec_np_b["TARGET"].unique())
print("Non Polar Faint  :",len(df_spec_np_f["TARGET"].unique()),"\t",df_spec_np_f["TARGET"].unique())

In [ ]:
#dn = GetNightBoundariesDict(df_spec_sel)

## Check saved hdf5 files

In [ ]:
list_of_hdf5_files = sorted(os.listdir(datapath_input))
list_of_hdf5_files

## Select photometric night 

In [ ]:
df_selection = df_selection[df_selection["flag_photom"]]
df_selection.drop(["flag_photom"],axis=1,inplace=True)
list_of_selected_nights = df_selection["nights"].unique()

In [ ]:
df_selection

In [ ]:
df_selection = df_selection[df_selection["flag_photom"]]
df_selection.drop(["flag_photom"],axis=1,inplace=True)
list_of_selected_nights = df_selection["nights"].unique()

In [ ]:
#print(list_of_selected_nights)

In [ ]:
list_of_hdf5_files_selected = []
list_of_hdf5_files_rejected = []
for file in list_of_hdf5_files:
    nightnumstr = re.findall("^spectra_transmission_(.*)[.]h5$",file)[0]
    nightnum = int(nightnumstr)
    if nightnum in list_of_selected_nights:
        list_of_hdf5_files_selected.append(file)
    else:
        list_of_hdf5_files_rejected.append(file) 

In [ ]:
list_of_hdf5_files_selected 

In [ ]:
list_of_hdf5_files_rejected 

In [ ]:
list_of_hdf5_files  = list_of_hdf5_files_selected 

## Per night transmission at airmass z=1

In [ ]:
#colormap = cm.jet 
#normalize = mcolors.Normalize(vmin=np.min(airmass), vmax=np.max(airmass))

In [ ]:
XMIN=880.
XMAX=1000.
XMINSEL = 900.
XMAXSEL=980.
FLAG_SHOWGREYPATCHS = True


all_figs_to_pdf = []
all_night_dict = []
all_night_dateobs = []

for idx_file,fname in enumerate(list_of_hdf5_files):
    ffname = os.path.join(datapath_input,fname)
    night_dict = readhdf5_pernightspectra(ffname)
    dateobs = int(re.findall("^spectra_transmission_(.*)[.]h5",fname)[0])

    all_night_dict.append(night_dict)
    all_night_dateobs.append(dateobs)

    # find the airmasses
    Nobs = len(night_dict)
    all_airmasses = np.zeros(Nobs)
    all_dt = np.zeros(Nobs)
    all_targets = []
    for idx,key in enumerate(night_dict.keys()):
        all_airmasses[idx]  = night_dict[key]['attr']['airmass']
        all_dt[idx] = night_dict[key]['attr']['dt_midnight']
        all_targets.append(night_dict[key]['attr']['target'])

    list_of_targets = np.array(all_targets)
    list_of_targets = np.unique(list_of_targets)
    textstr_targets = '\n'.join(list_of_targets)
    Ntargets = len(list_of_targets) 
        
    colormap = cm.jet 
    normalize = mcolors.Normalize(vmin=np.min(all_airmasses), vmax=np.max(all_airmasses))
    
    # Colorbar setup
    s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
    s_map.set_array(all_airmasses)

    zmin = all_airmasses.min()
    zmax = all_airmasses.max()
    
    textstr = '\n'.join((
    r'$N_{spec}=%.0f$' % (Nobs, ),
    f' {zmin:.2f} < airmass < {zmax:.2f}' ))


    all_colors = []
    
    #fig,ax = plt.subplots(1,1,figsize=(12,6))

    fig = plt.figure(figsize=(12,10),constrained_layout=True)
    gs = fig.add_gridspec(2, 1,height_ratios=[4,1.5])
    ax = fig.add_subplot(gs[0,0])
    ax1 = fig.add_subplot(gs[1,0])

    
    for key in night_dict.keys():

        the_airmass = night_dict[key]['attr']['airmass']
        the_color = colormap(normalize(the_airmass))
        all_colors.append(the_color) 
        
        the_datasets =  night_dict[key]["datasets"]
        wls = the_datasets['wls']
        transm = the_datasets['transm_atz12']
        ax.plot(wls,transm,color=the_color)
        transm =  the_datasets['transm_atz12']
        the_attributes =  night_dict[key]["attr"]
        
    ax.set_ylim(0,1.2)  

    if FLAG_SHOWGREYPATCHS:
        ax.set_xlim(XMIN,XMAX)  
        ax.axvspan(XMIN,XMINSEL, alpha=0.5, color='grey')
        ax.axvspan(XMAXSEL,XMAX, alpha=0.5, color='grey')
    else: 
        ax.set_xlim(XMINSEL,XMAXSEL)

    
    ax.grid()
    #ax.axvline(486.86,lw=1,color="k")
    #ax.axvline(656.279,lw=1,color="k")

    ax.text(0.1, 0.2, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    ax.text(0.78, 0.02*(Ntargets-1), textstr_targets,transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=props)  
    
    cbar = fig.colorbar(s_map,ax=ax)
    cbar.set_label("Airmass $z$")

    ax.axvline(490.00,lw=1,color="k",ls=":")
    ax.axvline(650.00,lw=1,color="k",ls=":")
    ax.set_xlabel("$\\lambda$ (nm)")
    ax.set_ylabel("transmission at z=1")
    ax.set_title(f"atmospheric transmission for night {dateobs}")


    ax1.scatter(all_dt,all_airmasses,marker="o",color=all_colors) 
    #ax1.set_ylim(zmax*1.1,zmin*0.9)
    ax1.set_ylim(2.6,0.9)
    ax1.set_ylabel("airmass")
    ax1.set_xlabel("time since midnight (hours)")
    ax1.grid()

    for idx, target in enumerate(all_targets):
        ax1.text(all_dt[idx], 2.5, target,color=all_colors[idx],rotation="vertical",fontsize=10)
    



    # save fig in fig container

    all_figs_to_pdf.append(fig) 

    # save individual figures
    figname =f"{pathfigs}/annnighttransmissionam1_{dateobs}"+figtype
    plt.savefig(figname)
      


    plt.show()

    #if idx_file == 0:
    #    break

      

## Transmission at Airmass Z

In [ ]:
#XMIN=900.
#XMAX=980.
#XMINSEL = 380.
#XMAXSEL=980.

XMIN=880.
XMAX=1000.
XMINSEL = 900.
XMAXSEL=980.

FLAG_SHOWGREYPATCHS = True


all_figs_to_pdf = []
all_night_dict = []
all_night_dateobs = []

for idx_file,fname in enumerate(list_of_hdf5_files):
    ffname = os.path.join(datapath_input,fname)
    night_dict = readhdf5_pernightspectra(ffname)
    dateobs = int(re.findall("^spectra_transmission_(.*)[.]h5",fname)[0])

    all_night_dict.append(night_dict)
    all_night_dateobs.append(dateobs)

    # find the airmasses
    Nobs = len(night_dict)
    all_airmasses = np.zeros(Nobs)
    all_dt = np.zeros(Nobs)
    all_targets = []
    for idx,key in enumerate(night_dict.keys()):
        all_airmasses[idx]  = night_dict[key]['attr']['airmass']
        all_dt[idx] = night_dict[key]['attr']['dt_midnight']
        all_targets.append(night_dict[key]['attr']['target'])

    list_of_targets = np.array(all_targets)
    list_of_targets = np.unique(list_of_targets)
    textstr_targets = '\n'.join(list_of_targets)
    Ntargets = len(list_of_targets) 
        
    colormap = cm.jet 
    normalize = mcolors.Normalize(vmin=np.min(all_airmasses), vmax=np.max(all_airmasses))
    
    # Colorbar setup
    s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
    s_map.set_array(all_airmasses)

    zmin = all_airmasses.min()
    zmax = all_airmasses.max()
    
    textstr = '\n'.join((
    r'$N_{spec}=%.0f$' % (Nobs, ),
    f' {zmin:.2f} < airmass < {zmax:.2f}' ))


    all_colors = []
    
    #fig,ax = plt.subplots(1,1,figsize=(12,6))

    fig = plt.figure(figsize=(12,10),constrained_layout=True)
    gs = fig.add_gridspec(2, 1,height_ratios=[4,1.5])
    ax = fig.add_subplot(gs[0,0])
    ax1 = fig.add_subplot(gs[1,0])

    
    for key in night_dict.keys():

        the_airmass = night_dict[key]['attr']['airmass']
        the_color = colormap(normalize(the_airmass))
        all_colors.append(the_color) 
        
        the_datasets =  night_dict[key]["datasets"]
        wls = the_datasets['wls']
        transm = the_datasets['transm_atz']
        ax.plot(wls,transm,color=the_color)
        transm =  the_datasets['transm_atz']
        the_attributes =  night_dict[key]["attr"]
        
    ax.set_ylim(0,1.2)  

    if FLAG_SHOWGREYPATCHS:
        ax.set_xlim(XMIN,XMAX)  
        ax.axvspan(XMIN,XMINSEL, alpha=0.5, color='grey')
        ax.axvspan(XMAXSEL,XMAX, alpha=0.5, color='grey')
    else: 
        ax.set_xlim(XMINSEL,XMAXSEL)

    
    ax.grid()
    #ax.axvline(486.86,lw=1,color="k")
    #ax.axvline(656.279,lw=1,color="k")

    ax.text(0.1, 0.2, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    ax.text(0.78, 0.02*(Ntargets-1), textstr_targets,transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=props)  
    
    cbar = fig.colorbar(s_map,ax=ax)
    cbar.set_label("Airmass $z$")

    ax.axvline(490.00,lw=1,color="k",ls=":")
    ax.axvline(650.00,lw=1,color="k",ls=":")
    ax.set_xlabel("$\\lambda$ (nm)")
    ax.set_ylabel("transmission at z")
    ax.set_title(f"atmospheric transmission for night {dateobs}")


    ax1.scatter(all_dt,all_airmasses,marker="o",color=all_colors) 
    #ax1.set_ylim(zmax*1.1,zmin*0.9)
    ax1.set_ylim(2.6,0.9)
    ax1.set_ylabel("airmass")
    ax1.set_xlabel("time since midnight (hours)")
    ax1.grid()

    for idx, target in enumerate(all_targets):
        ax1.text(all_dt[idx], 2.5, target,color=all_colors[idx],rotation="vertical",fontsize=10)
    



    # save fig in fig container

    all_figs_to_pdf.append(fig) 

    # save individual figures
    figname =f"{pathfigs}/annnighttransmissionamz_{dateobs}"+figtype
    plt.savefig(figname)
      


    plt.show()

    #if idx_file == 0:
    #    break

      

## Make BouguerLine

- Simpler than in https://github.com/sylvielsstfr/AuxTelComm/blob/main/notebooks_usdf/BouguerLines/202307/MLfit_bouguer_linearfit-OneBigBin-20230118-empty~holo_psf2dffm-rebin1.ipynb

In [ ]:
def MakeBouguerLinesFromTransmission(fname,wlcenter,wlwidth):
    """

    Simple method to do the Bouguer line: Compute the average transmission in the wavelength bin.
    Another method would integrate the SED-Flux and the Spectrum flux. To be done later

    Parameters:
        fname : hdf5 file containing the spectra
        wlcenter,wlwidth : wavelength bin for which compute wavelength
    """

    ffname = os.path.join(datapath_input,fname)
    night_dict = readhdf5_pernightspectra(ffname)
    dateobs = int(re.findall("^spectra_transmission_(.*)[.]h5",fname)[0])


    # find the airmasses
    Nobs = len(night_dict)
    all_airmasses = np.zeros(Nobs)
    all_pwv = np.zeros(Nobs)
    all_dt = np.zeros(Nobs)
    all_targets = []
    all_transm_mean =  np.zeros(Nobs)
    all_transm_std =  np.zeros(Nobs)
        

    # loop on observation
    for idx,key in enumerate(night_dict.keys()):
        all_airmasses[idx]  = night_dict[key]['attr']['airmass']
        pwv = night_dict[key]['attr']['PWV [mm]_x']
        all_pwv[idx] = pwv
        all_dt[idx] = night_dict[key]['attr']['dt_midnight']
        all_targets.append(night_dict[key]['attr']['target'])
        the_datasets =  night_dict[key]["datasets"]
        wls = the_datasets['wls']
        transm = the_datasets['transm_atz']
        indexes_wls = np.where(np.logical_and(wls>= wlcenter-wlwidth/2.,wls<wlcenter+wlwidth/2.))[0]
        transm_selected = transm[indexes_wls] 
        all_transm_mean[idx] = transm_selected.mean()
        all_transm_std[idx] = transm_selected.std()
        
        
    list_of_targets = np.array(all_targets)
    list_of_targets = np.unique(list_of_targets)
   
    Ntargets = len(list_of_targets) 
    return all_airmasses,all_pwv,all_transm_mean,all_transm_std, dateobs


In [ ]:
print(list_of_hdf5_files)

## Plot extinction

### Plot extinction per night

In [ ]:
am,pwv,y,ey, dateobs = MakeBouguerLinesFromTransmission('spectra_transmission_20241015.h5',940.,15.)
am,pwv,y,ey, dateobs = MakeBouguerLinesFromTransmission('spectra_transmission_20240925.h5',940.,15.)

In [ ]:
wlbins = np.arange(900.,980.,10)

In [ ]:
wlbins

In [ ]:
colormap = cm.jet 
normalize = mcolors.Normalize(vmin=np.min(wlbins), vmax=np.max(wlbins))
# Colorbar setup
#s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
#s_map.set_array(wlbins)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,5),constrained_layout=True)

zpwv = am*pwv
mag = -2.5*np.log10(y)
emag = ey/y

magmax = np.max(mag+emag)

ax1.errorbar(zpwv,mag,yerr=emag,fmt="o",color="r",ecolor="k")
ax1.set_xlim(0.,zpwv.max()*1.2)
ax1.set_ylim(0.,magmax)

ax2.errorbar(zpwv,mag,yerr=emag,fmt="o",color="r",ecolor="k")
ax2.set_xlim(0.1,zpwv.max()*1.2)
ax2.set_ylim(0,magmax)
ax2.set_xscale("log")

In [ ]:
fname = 'spectra_transmission_20240925.h5'

fig,axs = plt.subplots(2,2,figsize=(16,10),constrained_layout=True)

ax1,ax2,ax3,ax4 = axs.flat

all_am = np.array([])
all_zpwv = np.array([])
all_magmax = np.array([])

for idx_wl ,wlbin in enumerate(wlbins):
    the_color = colormap(normalize(wlbin))
    am,pwv,y,ey,dateobs = MakeBouguerLinesFromTransmission(fname,wlbin,15.)

    label = f"$\lambda = $" +f"{wlbin:.1f} nm"

    zpwv = am*pwv
    mag = -2.5*np.log10(y)
    emag = ey/y
    magmax = np.max(mag+emag)
    
    all_am = np.append(all_am,am)
    all_zpwv = np.append(all_zpwv,zpwv)
    all_magmax = np.append(all_magmax,magmax)

    ax1.errorbar(am,mag,yerr=emag,fmt="o",color=the_color,ecolor="k",label=label)
    ax2.errorbar(am,mag,yerr=emag,fmt="o",color=the_color,ecolor="k")
    

    ax3.errorbar(zpwv,mag,yerr=emag,fmt="o",color=the_color,ecolor="k")
    
    ax4.errorbar(zpwv,mag,yerr=emag,fmt="o",color=the_color,ecolor="k")

ax1.set_xlim(0.,all_am.max()*1.2)
ax1.set_ylim(0.,all_magmax.max())
ax1.set_xlabel("airmass")
ax1.set_ylabel("$-2.5\log_{10} (T_\\lambda)$")
ax1.grid()
ax1.legend()

ax2.set_xlim(0.1,all_am.max()*1.2)
ax2.set_ylim(0.,all_magmax.max())
ax2.set_xlabel("airmass (log scale)")
ax2.set_xscale("log")
ax2.grid()

ax3.set_xlim(0.,all_zpwv.max()*1.2)
ax3.set_ylim(0.,all_magmax.max())
ax3.set_xlabel("airmass x pwv (mm)")
ax3.set_ylabel("$-2.5\log_{10} (T_\\lambda)$")
ax3.grid()

ax4.set_xlim(0.1,all_zpwv.max()*1.2)
ax4.set_ylim(0.,all_magmax.max())
ax4.set_xscale("log")
ax4.set_xlabel("airmass x pwv (mm) (log scale)")
ax4.grid()

suptitle = f"Bouguer curves in PWV region night = {dateobs}"
plt.suptitle(suptitle,fontsize=20,fontweight='bold')
plt.show()

In [ ]:
def PlotBouguerlineCurves(fname,wlbins,wlwidth):
    """
    """
    fig,axs = plt.subplots(2,2,figsize=(16,10),constrained_layout=True)

    ax1,ax2,ax3,ax4 = axs.flat

    all_am = np.array([])
    all_zpwv = np.array([])
    all_magmax = np.array([])

    for idx_wl ,wlbin in enumerate(wlbins):
        the_color = colormap(normalize(wlbin))
        am,pwv,y,ey,dateobs = MakeBouguerLinesFromTransmission(fname,wlbin,wlwidth)

        label = f"$\lambda = $" +f"{wlbin:.1f} nm"

        zpwv = am*pwv
        mag = -2.5*np.log10(y)
        emag = ey/y
        magmax = np.max(mag+emag)
    
        all_am = np.append(all_am,am)
        all_zpwv = np.append(all_zpwv,zpwv)
        all_magmax = np.append(all_magmax,magmax)

        ax1.errorbar(am,mag,yerr=emag,fmt="o",color=the_color,ecolor="k",label=label)
        ax2.errorbar(am,mag,yerr=emag,fmt="o",color=the_color,ecolor="k")
        ax3.errorbar(zpwv,mag,yerr=emag,fmt="o",color=the_color,ecolor="k")
        ax4.errorbar(zpwv,mag,yerr=emag,fmt="o",color=the_color,ecolor="k")

    ax1.set_xlim(0.,all_am.max()*1.2)
    ax1.set_ylim(0.,all_magmax.max())
    ax1.set_xlabel("airmass")
    ax1.set_ylabel("$-2.5\log_{10} (T_\\lambda)$")
    ax1.grid()
    ax1.legend()

    ax2.set_xlim(0.1,all_am.max()*1.2)
    ax2.set_ylim(0.,all_magmax.max())
    ax2.set_xlabel("airmass (log scale)")
    ax2.set_xscale("log")
    ax2.grid()

    ax3.set_xlim(0.,all_zpwv.max()*1.2)
    ax3.set_ylim(0.,all_magmax.max())
    ax3.set_xlabel("airmass x pwv (mm)")
    ax3.set_ylabel("$-2.5\log_{10} (T_\\lambda)$")
    ax3.grid()

    ax4.set_xlim(0.1,all_zpwv.max()*1.2)
    ax4.set_ylim(0.,all_magmax.max())
    ax4.set_xscale("log")
    ax4.set_xlabel("airmass x pwv (mm) (log scale)")
    ax4.grid()

    suptitle = f"Bouguer curves in PWV region night = {dateobs}"
    plt.suptitle(suptitle,fontsize=20,fontweight='bold')
    plt.show()

In [ ]:
wlbins = np.arange(900.,980.,10)
wlwidth = 15
for idx_file,fname in enumerate(list_of_hdf5_files):
    PlotBouguerlineCurves(fname,wlbins,wlwidth)
    

In [ ]:
def ExtractBouguerlineCurves(fname,wlbins,wlwidth):
    """
    Extract Bouguer lineCurves (Not plot)

    Parameters
       fname : filename
       wlbins : array of wavelengths
       wlwidth : bin size
    """


    all_am = []
    all_pwv = []
    all_zpwv = []
    all_mag = []
    all_emag = []
    all_dateobs = []

    # loop on wavelengths bins
    for idx_wl ,wlbin in enumerate(wlbins):
        the_color = colormap(normalize(wlbin))
        am,pwv,y,ey,dateobs = MakeBouguerLinesFromTransmission(fname,wlbin,wlwidth)

        zpwv = am*pwv
        mag = -2.5*np.log10(y)
        emag = ey/y
    
        all_am.append(am)
        all_pwv.append(pwv)
        all_zpwv.append(zpwv)
        all_mag.append(mag)
        all_emag.append(emag)
        all_dateobs.append(dateobs)

    return all_am,all_pwv,all_zpwv,all_mag,all_emag,all_dateobs


### Plot Extinction cumulated over night

#### Plot Extinction cumulated over night wrt airmass

In [ ]:
import itertools
marker = itertools.cycle(('o', '+', '^', 'v', '*', '>' , '<', 's', 'P', 'h', 'X', 'D')) 

In [ ]:
#fig,(ax1,ax2) = plt.subplots(2,1,figsize=(16,16),constrained_layout=True)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6),constrained_layout=True)

wlbins = np.arange(900.,980.,10)
wlwidth = 20.
wlhalfwidth =  wlwidth/2.
colormap = cm.jet 
normalize = mcolors.Normalize(vmin=np.min(wlbins), vmax=np.max(wlbins))



for idx_file,fname in enumerate(list_of_hdf5_files):
    all_am,all_pwv,all_zpwv,all_mag,all_emag,all_dateobs = ExtractBouguerlineCurves(fname,wlbins,wlwidth)


    for idx_wl ,wlbin in enumerate(wlbins):
        the_color = colormap(normalize(wlbin))
        the_marker = next(marker)
        the_label = "$\\lambda = $" +f"{wlbin:.0f}" + "$\pm$" + f"{wlhalfwidth:.0f} nm"
        
        am = all_am[idx_wl]
        pwv = all_pwv[idx_wl]
        zpwv = all_zpwv[idx_wl]
        mag = all_mag[idx_wl]
        emag = all_emag[idx_wl]

        
        if idx_file==0:
            ax1.errorbar(am,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="k",label=the_label)
            ax2.errorbar(am,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="k",label=the_label)
        else:
            ax1.errorbar(am,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="k")
            ax2.errorbar(am,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="k")

xmin,xmax = ax1.get_xlim()
ymin,ymax = ax1.get_ylim()

ax1.legend(loc="upper left")
ax1.set_xlim(1.,xmax)
ax1.set_ylim(0.,ymax)
ax1.set_xlabel("airmass")
ax1.set_ylabel("$-2.5\log_{10} (T_\\lambda)^{meas} = \\tau_{\lambda}^{meas}$")
ax1.grid()

 

xmin,xmax = ax2.get_xlim()
ymin,ymax = ax2.get_ylim()

ax2.legend(loc="upper left")
ax2.set_xlim(1.,xmax)
ax2.set_ylim(0,ymax)
ax2.set_xlabel("airmass (log scale)")
ax2.set_ylabel("$-2.5\log_{10} (T_\\lambda)^{meas} = \\tau_{\lambda}^{meas}$")
ax2.set_xscale("log")
ax2.grid()

suptitle ="Measured Extinction vs $airmass$"
plt.suptitle(suptitle,fontsize=20,fontweight="bold")
plt.show()

#### Plot Extinction cumulated over night wrt airmass x PWV

In [ ]:
marker = itertools.cycle(('o', '+', '^', 'v', '*', '>' , '<', 's', 'P', 'h', 'X', 'D')) 

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6),constrained_layout=True)

wlbins = np.arange(900.,980.,10)
wlwidth = 20.
wlhalfwidth =  wlwidth/2.
colormap = cm.jet 
normalize = mcolors.Normalize(vmin=np.min(wlbins), vmax=np.max(wlbins))


for idx_file,fname in enumerate(list_of_hdf5_files):
    all_am,all_pwv,all_zpwv,all_mag,all_emag,all_dateobs = ExtractBouguerlineCurves(fname,wlbins,wlwidth)


    for idx_wl ,wlbin in enumerate(wlbins):
        the_color = colormap(normalize(wlbin))
        the_marker = next(marker)
        the_label = "$\\lambda = $" +f"{wlbin:.0f}" + "$\pm$" + f"{wlhalfwidth:.0f} nm"
        
        am = all_am[idx_wl]
        pwv = all_pwv[idx_wl]
        zpwv = all_zpwv[idx_wl]
        mag = all_mag[idx_wl]
        emag = all_emag[idx_wl]

        
        if idx_file==0:
            ax1.errorbar(zpwv,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="grey",label=the_label)
            ax2.errorbar(zpwv,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="grey",label=the_label)
        else:
            ax1.errorbar(zpwv,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="grey")
            ax2.errorbar(zpwv,mag,yerr=emag,fmt=the_marker,color=the_color,ecolor="grey")

xmin,xmax = ax1.get_xlim()
ymin,ymax = ax1.get_ylim()

ax1.legend(loc="upper left")
ax1.set_xlim(0.,xmax*1.1)
ax1.set_ylim(0.,ymax*1.1)
ax1.set_xlabel("$airmass \\times pwv (mm)$")
ax1.set_ylabel("$-2.5\log_{10} (T_\\lambda)^{meas} = \\tau_{\lambda}^{meas}$")
ax1.grid()

 

xmin,xmax = ax2.get_xlim()
ymin,ymax = ax2.get_ylim()

ax2.legend(loc="upper left")
ax2.set_xlim(1.,xmax*1.1)
ax2.set_ylim(0,ymax)
ax2.set_xlabel("$airmass \\times pwv (mm)$ (log scale)")
ax2.set_ylabel("$-2.5\log_{10} (T_\\lambda)^{meas} = \\tau_{\lambda}^{meas}$")
ax2.set_xscale("log")
ax2.grid()

suptitle ="Measured Extinction vs $airmass \\times PWV$"
plt.suptitle(suptitle,fontsize=20,fontweight="bold")
plt.show()

In [ ]:
assert False

### Save all transmission curves 

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
pdf_filename = f"holo_night_transmissionsz_tight.pdf"
pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
with PdfPages(pdf_fullfilename) as pdf:
    for fig in all_figs_to_pdf:
        pdf.savefig(fig, bbox_inches='tight') 

In [ ]:
pdf_filename = f"holo_night_transmissionsz_nottight.pdf"
pdf_fullfilename = os.path.join(pathfigs,pdf_filename)
with PdfPages(pdf_fullfilename) as pdf:
    for fig in all_figs_to_pdf:
        pdf.savefig(fig) 